In [ ]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2

In [ ]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbidden_area_score=-1, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()

In [ ]:
class MC_epsilon_greedy:
    """
    Monte Carlo Epsilon-Greedy Agent.

    This class represents an agent that uses Monte Carlo control with epsilon-greedy exploration to learn an optimal policy.

    Attributes:
        gamma (float): The discount factor for future rewards.
        epsilon (float): The exploration rate, determining the probability of taking a random action.
        Q_table (numpy.ndarray): The Q-table storing the action-value estimates for each state-action pair.
        Q_table_num (numpy.ndarray): The table storing the number of times each state-action pair has been visited.
        Q_table_reward (numpy.ndarray): The table storing the cumulative rewards for each state-action pair.

    Methods:
        take_action(state): Takes an action based on the epsilon-greedy policy.
        update(episode): Updates the Q-table based on the rewards obtained in an episode.
    """

    def __init__(self, rows, columns, gamma, epsilon):
        """
        Initializes a new instance of the MC_epsilon_greedy class.

        Args:
            rows (int): The number of rows in the environment.
            columns (int): The number of columns in the environment.
            gamma (float): The discount factor for future rewards.
            epsilon (float): The exploration rate, determining the probability of taking a random action.
        """
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q_table = np.zeros((rows, columns, 5))
        self.Q_table_num = np.zeros((rows, columns, 5))
        self.Q_table_reward = np.zeros((rows, columns, 5))

    def take_action(self, state):
        """
        Takes an action based on the epsilon-greedy policy.

        Args:
            state (tuple): The current state of the agent.

        Returns:
            int: The selected action.
        """
        if np.random.random() < self.epsilon:
            action = np.random.randint(0, 5)
        else:
            action = np.argmax(self.Q_table[state])
        return action

    def update(self, episode):
        """
        Updates the Q-table based on the rewards obtained in an episode.

        Args:
            episode (list): The list of (state, action, reward) tuples representing an episode.
        """
        G = 0
        for i in reversed(episode):
            state, action, score = i
            G = self.gamma * G + score
            self.Q_table_reward[state][action] += G
            self.Q_table_num[state][action] += 1
            self.Q_table[state][action] = self.Q_table_reward[state][action] / self.Q_table_num[state][action]
class MC_epsilon_greedy:
    def __init__(self,rows,columns, gamma, epsilon):
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q_table = np.zeros((rows,columns,5))
        self.Q_table_num = np.zeros((rows,columns,5))
        self.Q_table_reward = np.zeros((rows,columns,5))

    def take_action(self,state):
        if np.random.random() < self.epsilon:
            action = np.random.randint(0,5)
        else:
            action = np.argmax(self.Q_table[state])
        return action
    
    def update(self,episode):
        G = 0
        for i in reversed(episode):
            state, action, score = i
            G = self.gamma*G + score
            self.Q_table_reward[state][action] += G
            self.Q_table_num[state][action] += 1
            # 不能直接除,因为有0
            self.Q_table[state][action] = self.Q_table_reward[state][action]/self.Q_table_num[state][action]
            # self.Q_table[] = self.Q_table_reward/self.Q_table_num
        

In [ ]:
np.random.seed(0)
epsilon = 0.9
gamma = 0.9
agent = MC_epsilon_greedy(rows,columns,gamma,epsilon)
num_episodes = 20000

In [ ]:
gridworld.show_policy(np.argmax(agent.Q_table,axis=2))

In [ ]:
# for i in range(num_episodes): 
#     state = gridworld.reset((0,0))
#     trajectory = []
#     for _ in range(10000):
#         action = agent.take_action(state)
#         next_state,score,done = gridworld.step(state,action)
#         trajectory.append((state,action,score))
#         state = next_state
    
#     agent.update(trajectory)
#     gridworld.show_policy(agent.Q_table.argmax(axis=2),step=False)
    # print(agent.Q_table)
    # print(agent.Q_table_reward)
    # print(agent.Q_table_num)
    # break

In [ ]:
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))].reshape(rows,columns,5) 
qtable = np.zeros((rows*columns,5)).reshape(rows,columns,5)  #生成Qtable，也就是action-value-table
# gridworld.show()
# gridworld.show_policy(policy)
# print("random policy")

trajectorySteps = 50000

epsilon = 0.1 #0.2


num_episodes = 200

values = np.ones(rows*columns).reshape(rows,columns)       #初始化可以任意，也可以全0
pre_values = np.zeros(rows*columns).reshape(rows,columns)       #初始化可以任意，也可以全0
for episode in range(num_episodes):
    # 
    if(epsilon > 0.001) :
        epsilon -= 0.001
    else:
        epsilon = 0.001
        
    p1 = 1-epsilon * (4/5)
    p0 = epsilon/5
    # trajectorySteps = int(20+epsilon*1000)
    print("trajectorySteps",trajectorySteps)
    print(f"epision:{epsilon}, p1:{p1}, p0:{p0}")
    
    d = {1:p1, 0:p0}
    policy_epsilon = np.vectorize(d.get)(policy)
    

    cnt = np.zeros(25).reshape(rows,columns)
    qtable_rewards = np.zeros((rows,columns,5))
    qtable_nums = np.zeros((rows,columns,5))
    i = random.randint(0,24)  #初始状态
    j = random.randint(0,4)
    # 不是正方形会有bug
    Trajectory = gridworld.get_trajectory_score(now_state=(i//rows,i%rows), action=j, policy=policy_epsilon, steps=trajectorySteps)
    # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
    score = 0
    # for k in range(trajectorySteps,-1,-1):
    for k in reversed(Trajectory):
        tmpstate, tmpaction, tmpscore, _, __  = k
        # cnt[tmpstate] += 1
        score = score*gamma + tmpscore  #细节从后往前优化算法
        
        qtable_rewards[tmpstate][tmpaction] += score
        qtable_nums[tmpstate][tmpaction] += 1
        qtable[tmpstate][tmpaction] = qtable_rewards[tmpstate][tmpaction] / qtable_nums[tmpstate][tmpaction]

    values = (qtable*policy_epsilon).sum(axis=2)
    
    
    policy = np.eye(5)[np.argmax(qtable,axis=2).flatten()].reshape(rows,columns,5)  #qtable的最优值作为更新策略，并用独热码来表示
    gridworld.show_policy(policy.argmax(axis=2),step=False)

    gridworld.show_value(values)
    policy_epsilon = np.vectorize(d.get)(policy)
    if(np.sum(np.abs(values-pre_values)) < 0.001):
        print(f"{episode=}")
        print("converge")
        break
        
    # print(np.array(cnt).reshape(5,5))


In [ ]:
# values = np.ones(rows*columns).reshape(rows,columns)       #初始化可以任意，也可以全0
# gridworld.show_value(values)